# Example

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
from matplotlib import pyplot as plt
import matplotlib
% matplotlib inline
from subprocess import call, Popen, PIPE

from madminer import GoldMine, Smithy
from madminer.tools.plots import plot_2d_morphing_basis
from delphesminer import DelphesMiner

## 1. Define parameter space

In [2]:
miner = GoldMine()

miner.add_parameter(
    lha_block='ANOINPUTS',
    lha_id=3,
    parameter_name='fW',
    morphing_max_power=4,
    parameter_range=(-1.2e-05,1.2e-05)
)
miner.add_parameter(
    lha_block='ANOINPUTS',
    lha_id=7,
    parameter_name='fWW',
    morphing_max_power=4,
    parameter_range=(-1.2e-05,1.2e-05)
)

## 2. Define benchmark points (evaluation points for |M|^2)

### 2a. Set benchmarks by hand

In [ ]:
miner.add_benchmark(
    {'fW':0., 'fWW':0.},
    'sm'
)

### 2b. Optimize benchmarks for morphing

In [ ]:
miner.set_benchmarks_from_morphing(
    keep_existing_benchmarks=True,
    n_bases=1,
    n_trials=1000,
    n_test_thetas=100
)

In [ ]:
fig = plot_2d_morphing_basis(
    miner.morpher,
    xlabel=r'$f_{W} / \Lambda^2$',
    ylabel=r'$f_{WW} / \Lambda^2$ [GeV${}^{-2}$]',
    xrange=(-1.64949627e-05, 1.64949627e-05),
    yrange=(-1.64949627e-05, 1.64949627e-05),
)

## 3. Save settings and run MadGraph

In [ ]:
miner.save('data/madminer_example.h5')

In [ ]:
miner.run(
    mg_directory='/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2',
    proc_card_file='./cards/proc_card.dat',
    param_card_template_file='./cards/param_card_template.dat',
    reweight_card_template_file='./cards/reweight_card_template.dat',
    run_card_file='./cards/run_card.dat',
    sample_benchmark='sm',
    initial_command='source activate python2'
)

## 4. Run detector simulation and extract observables

To increase flexibility, this is not part of the madminer package. The delphesminer package includes the necessary steps based on Delphes

In [3]:
dm = DelphesMiner()

In [4]:
dm.add_hepmc_sample('./MG_process/Events/run_01/tag_1_pythia8_events.hepmc.gz')

In [5]:
dm.run_delphes(delphes_directory='/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/Delphes',
               delphes_card='./cards/delphes_card.dat')

In [6]:
dm.add_observable('n_jets', 'len(j)')
dm.add_observable('pt_mu1', 'mu[0].pt')
dm.add_observable('pt_mu2', 'mu[1].pt')
dm.add_observable('pt_j1', 'j[0].pt')
dm.add_observable('pt_j2', 'j[1].pt')
dm.add_observable('m_jj', '(j[0] + j[1]).m')
dm.add_observable('delta_eta_jj', 'abs(j[0].eta() - j[1].eta())')
dm.add_observable('delta_phi_jj', 'abs(j[0].phi() - j[1].phi())')

In [7]:
dm.analyse_delphes_samples()

KeyError: "not found: b'Weight.Weight'"

In [ ]:
dm.save('data/madminer_example_with_data.h5', 'data/madminer_example.h5')

In [8]:
import uproot

root_file = uproot.open(dm.delphes_sample_filenames[0])
tree = root_file['Delphes']
tree.keys()

[b'Event',
 b'Event_size',
 b'Particle',
 b'Particle_size',
 b'Track',
 b'Track_size',
 b'Tower',
 b'Tower_size',
 b'EFlowTrack',
 b'EFlowTrack_size',
 b'EFlowPhoton',
 b'EFlowPhoton_size',
 b'EFlowNeutralHadron',
 b'EFlowNeutralHadron_size',
 b'GenJet',
 b'GenJet_size',
 b'GenMissingET',
 b'GenMissingET_size',
 b'Jet',
 b'Jet_size',
 b'Electron',
 b'Electron_size',
 b'Photon',
 b'Photon_size',
 b'Muon',
 b'Muon_size',
 b'FatJet',
 b'FatJet_size',
 b'MissingET',
 b'MissingET_size',
 b'ScalarHT',
 b'ScalarHT_size']

## 5. Make training and test samples

In [ ]:
smithy = Smithy('data/madminer_example_with_data.h5')

In [ ]:
print(smithy.parameters)
print(miner.parameters == smithy.parameters)
fig = plot_2d_morphing_basis(smithy.morpher, xlabel=r'$f_{W}$', ylabel=r'$f_{WW}$')

In [ ]:
theta, x, gold = smithy.extract_sample(theta_sampling_types=['benchmark'],
                                       theta_sampling_values=[0],
                                       n_samples_per_theta=10,
                                       augmented_data_definitions=[('ratio', 'benchmark', 1, 'benchmark', 0)])